In [37]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [38]:
# Load dataset
df = pd.read_csv("../sim_og.csv")

In [39]:
X = df[['a', 'q', 'delta_e']].values
y = df[['Cd']].values

In [40]:
max_epochs = 5000
hidden_neurons = 20
learning_rate = 0.01
activation_name = 'ReLU'
optimizer_name = 'Adam'

In [41]:
# Data split (random_state changed every run for variation)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1111, random_state=42)

In [42]:
# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val   = torch.tensor(X_val, dtype=torch.float32)
y_val   = torch.tensor(y_val, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)
y_test  = torch.tensor(y_test, dtype=torch.float32)

In [43]:
class CFFN(nn.Module):
    def __init__(self):
        super(CFFN, self).__init__()
        self.hidden = nn.Linear(3, 20)
        self.relu = nn.ReLU()
        self.out_from_hidden = nn.Linear(20, 1)
        self.out_from_source = nn.Linear(3, 1)

    def forward(self, x):
        out = self.hidden(x)
        out = self.relu(out)
        out_h = self.out_from_hidden(out)
        out_s = self.out_from_source(x)
        out_final = out_s + out_h
        return out_final

In [44]:
model = CFFN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [45]:
#train the model
actual_epoch = 0

for epoch in range(max_epochs):
    model.train()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    actual_epoch = epoch + 1
    
    #validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch+1}/{max_epochs}, Train Loss: {loss.item():}, Val Loss: {val_loss.item():}")    

Epoch 1000/5000, Train Loss: 8.063600398600101e-05, Val Loss: 8.743775106268004e-05
Epoch 2000/5000, Train Loss: 2.0439911168068647e-05, Val Loss: 2.2296573661151342e-05
Epoch 3000/5000, Train Loss: 9.63415459409589e-06, Val Loss: 1.060377780959243e-05
Epoch 4000/5000, Train Loss: 5.769026756752282e-06, Val Loss: 6.544694315380184e-06
Epoch 5000/5000, Train Loss: 4.004110451205634e-06, Val Loss: 4.3160530367458705e-06


In [46]:
# Test the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    mse = ((y_test - test_outputs) ** 2).mean().item()
    r2 = r2_score(y_test, test_outputs)
print(f'R2 = {r2}')
print(f"\nTraining stopped at epoch {actual_epoch}")
print(f"Final Train MSE: {loss.item():}")
print(f"Validation MSE: {val_loss.item():}")
print(f"Test MSE: {test_loss.item():}")

R2 = 0.9999463507711245

Training stopped at epoch 5000
Final Train MSE: 4.004110451205634e-06
Validation MSE: 4.3160530367458705e-06
Test MSE: 4.469975010579219e-06


In [47]:
# Save the trained model
torch.save(model.state_dict(), "trained_cffn.pth")
print("Model saved as trained_cffn.pth")

Model saved as trained_cffn.pth
